In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

%matplotlib inline

In [9]:
# read the sequences file
arquivo = 'hikoyu_fasta.txt'

file_content = open(arquivo, "r")
"""file_content.close() """

all_lines = file_content.readlines();

sequences = {}
sequences_info = {}
description = ''

for line in all_lines:
    
    sequence = []
    
    if ">" in line:
        description = line
        sequence = []
        sequences[description] = sequence
    else:
        sequences[description].append(line)


In [10]:
'''sequences['>MT126808.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SP02/human/2020/BRA, complete genome\n']'''

"sequences['>MT126808.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SP02/human/2020/BRA, complete genome\n']"

In [11]:
#count A,T,C,G,AA,AT,AC,AG,TT,TA,TC,TG,CC,CA,CT,CG,GG,GA,GT,GC for each sequence

for description in sequences:
    
    sequences_info[description] = {}
    
    sequences_info[description]['A'] = 0
    sequences_info[description]['T'] = 0
    sequences_info[description]['C'] = 0
    sequences_info[description]['G'] = 0
    
    sequences_info[description]['AA'] = 0
    sequences_info[description]['AT'] = 0
    sequences_info[description]['AC'] = 0
    sequences_info[description]['AG'] = 0
    
    sequences_info[description]['TA'] = 0
    sequences_info[description]['TT'] = 0
    sequences_info[description]['TC'] = 0
    sequences_info[description]['TG'] = 0
    
    sequences_info[description]['CA'] = 0
    sequences_info[description]['CT'] = 0
    sequences_info[description]['CC'] = 0
    sequences_info[description]['CG'] = 0
    
    sequences_info[description]['GA'] = 0
    sequences_info[description]['GT'] = 0
    sequences_info[description]['GC'] = 0
    sequences_info[description]['GG'] = 0
    
    sequences_info[description]['SIZE'] = 0
    
    #key = '>MT126808.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SP02/human/2020/BRA, complete genome\n'

    for line in sequences[description]:
        
        sequences_info[description]['SIZE'] += len(line)
        sequences_info[description]['A'] += line.count('A')
        sequences_info[description]['T'] += line.count('T')
        sequences_info[description]['C'] += line.count('C')
        sequences_info[description]['G'] += line.count('G')
        
        sequences_info[description]['AA'] += line.count('AA')
        sequences_info[description]['AT'] += line.count('AT')
        sequences_info[description]['AC'] += line.count('AC')
        sequences_info[description]['AG'] += line.count('AG')
    
        sequences_info[description]['TA'] += line.count('AT')
        sequences_info[description]['TT'] += line.count('TT')
        sequences_info[description]['TC'] += line.count('TC')
        sequences_info[description]['TG'] += line.count('TG')
    
        sequences_info[description]['CA'] += line.count('CA')
        sequences_info[description]['CT'] += line.count('CT')
        sequences_info[description]['CC'] += line.count('CC')
        sequences_info[description]['CG'] += line.count('CG')
    
        sequences_info[description]['GA'] += line.count('GA')
        sequences_info[description]['GT'] += line.count('GT')
        sequences_info[description]['GC'] += line.count('GC')
        sequences_info[description]['GG'] += line.count('GG')   
    

In [12]:
#calc nucleotideo bias

for description in sequences:
         
    fA = sequences_info[description]['A'] / sequences_info[description]['SIZE'] 
    fT = sequences_info[description]['T'] / sequences_info[description]['SIZE']
    fC = sequences_info[description]['C'] / sequences_info[description]['SIZE']
    fG = sequences_info[description]['G'] / sequences_info[description]['SIZE'] 
        
    fAA = sequences_info[description]['AA'] / sequences_info[description]['SIZE']       
    fAT = sequences_info[description]['AT'] / sequences_info[description]['SIZE']       
    fAC = sequences_info[description]['AC'] / sequences_info[description]['SIZE']       
    fAG = sequences_info[description]['AG'] / sequences_info[description]['SIZE']    
      
    fTA = sequences_info[description]['TA'] / sequences_info[description]['SIZE']       
    fTT = sequences_info[description]['TT'] / sequences_info[description]['SIZE']       
    fTC = sequences_info[description]['TC'] / sequences_info[description]['SIZE']       
    fTG = sequences_info[description]['TG'] / sequences_info[description]['SIZE']       

    fCA = sequences_info[description]['CA'] / sequences_info[description]['SIZE']       
    fCT = sequences_info[description]['CT'] / sequences_info[description]['SIZE']       
    fCC = sequences_info[description]['CC'] / sequences_info[description]['SIZE']       
    fCG = sequences_info[description]['CG'] / sequences_info[description]['SIZE']       

    fGA = sequences_info[description]['GA'] / sequences_info[description]['SIZE']       
    fGT = sequences_info[description]['GT'] / sequences_info[description]['SIZE']       
    fGC = sequences_info[description]['GC'] / sequences_info[description]['SIZE']       
    fGG = sequences_info[description]['GG'] / sequences_info[description]['SIZE']
        
    sequences_info[description]['fA'] = fA
    sequences_info[description]['fT'] = fT
    sequences_info[description]['fC'] = fC
    sequences_info[description]['fG'] = fG
       
    sequences_info[description]['pAA'] = fAA/fA*fA
    sequences_info[description]['pAT'] = fAT/fA*fT
    sequences_info[description]['pAC'] = fAC/fA*fC
    sequences_info[description]['pAG'] = fAC/fA*fG

    sequences_info[description]['pTA'] = fTA/fT*fA
    sequences_info[description]['pTT'] = fTT/fT*fT
    sequences_info[description]['pTC'] = fTC/fT*fC
    sequences_info[description]['pTG'] = fTC/fT*fG
      
    sequences_info[description]['pCA'] = fCA/fC*fA
    sequences_info[description]['pCT'] = fCT/fC*fT
    sequences_info[description]['pCC'] = fCC/fC*fC
    sequences_info[description]['pCG'] = fCC/fC*fG

    sequences_info[description]['pGA'] = fGA/fG*fA
    sequences_info[description]['pGT'] = fGT/fG*fT
    sequences_info[description]['pGC'] = fGC/fG*fC
    sequences_info[description]['pGG'] = fGC/fG*fG


In [13]:
sequences_info['>LC521925.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/AI/I-004/2020 RNA, complete genome\n']

{'A': 8920,
 'T': 9589,
 'C': 5486,
 'G': 5853,
 'AA': 2136,
 'AT': 2273,
 'AC': 1990,
 'AG': 1707,
 'TA': 2273,
 'TT': 2412,
 'TC': 1385,
 'TG': 2538,
 'CA': 2046,
 'CT': 2035,
 'CC': 772,
 'CG': 433,
 'GA': 1587,
 'GT': 1950,
 'GC': 1156,
 'GG': 959,
 'SIZE': 30346,
 'fA': 0.2939431885586239,
 'fT': 0.31598892770052067,
 'fC': 0.18078165161800566,
 'fG': 0.19287550253740196,
 'pAA': 0.07038818954722204,
 'pAT': 0.08052049693534567,
 'pAC': 0.04033133259190934,
 'pAG': 0.043029400229756716,
 'pTA': 0.0696770119505425,
 'pTT': 0.07948329269096421,
 'pTC': 0.026111438887364464,
 'pTG': 0.02785823036962162,
 'pCA': 0.10962591392470736,
 'pCT': 0.11721426683750631,
 'pCC': 0.02543992618467014,
 'pCG': 0.02714179510734129,
 'pGA': 0.07970063902999079,
 'pGT': 0.10527565505142925,
 'pGC': 0.03570538002228166,
 'pGG': 0.038093982732485335}

In [14]:
#creating a dataframe
df_original = pd.DataFrame.from_dict(sequences_info, orient='index', columns=['fA', 'fT', 'fC', 'fG',
                                                                     'pAA','pAT', 'pAC', 'pAG',
                                                                     'pTA','pTA', 'pTC', 'pTG',
                                                                     'pCA','pCT', 'pCC', 'pCG',
                                                                     'pGA','pGT', 'pGC', 'pGG'])

In [15]:
df_original.head()

,fA,fT,fC,fG,pAA,pAT,pAC,pAG,pTA,pTA,pTC,pTG,pCA,pCT,pCC,pCG,pGA,pGT,pGC,pGG
">LC521925.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/AI/I-004/2020 RNA, complete genome\n",0.293943,0.315989,0.180782,0.192876,0.070388,0.080520,0.040331,0.043029,0.069677,0.069677,0.026111,0.027858,0.109626,0.117214,0.025440,0.027142,0.079701,0.105276,0.035705,0.038094
">LC522972.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/KY/V-029/2020 RNA, complete genome\n",0.294015,0.315874,0.180702,0.193014,0.070022,0.080215,0.040183,0.042921,0.069497,0.069497,0.025895,0.027660,0.109646,0.116878,0.025514,0.027252,0.080035,0.105435,0.035752,0.038188
">LC522973.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-012/2020 RNA, complete genome\n",0.294015,0.315874,0.180702,0.193014,0.070022,0.080180,0.040203,0.042942,0.069467,0.069467,0.025895,0.027660,0.109753,0.116820,0.025514,0.027252,0.080035,0.105543,0.035690,0.038122
">LC522974.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-501/2020 RNA, complete genome\n",0.294015,0.315841,0.180735,0.193014,0.070022,0.080172,0.040211,0.042942,0.069474,0.069474,0.025903,0.027663,0.109733,0.116844,0.025514,0.027247,0.080035,0.105478,0.035728,0.038155
">LC522975.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-521/2020 RNA, complete genome\n",0.294015,0.315874,0.180735,0.192981,0.070022,0.080215,0.040211,0.042935,0.069497,0.069497,0.025900,0.027655,0.109733,0.116856,0.025514,0.027242,0.080049,0.105507,0.035734,0.038155


In [16]:
df_original.describe()

,fA,fT,fC,fG,pAA,pAT,pAC,pAG,pTA,pTA,pTC,pTG,pCA,pCT,pCC,pCG,pGA,pGT,pGC,pGG
count,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000
mean,0.292453,0.314068,0.179923,0.192076,0.069724,0.079866,0.040060,0.042766,0.069251,0.069251,0.026102,0.027865,0.108779,0.116579,0.025290,0.026998,0.079277,0.104841,0.035342,0.037729
std,0.012380,0.013531,0.007245,0.008001,0.003013,0.003606,0.001563,0.001729,0.003021,0.003021,0.000985,0.001095,0.004630,0.005144,0.001057,0.001164,0.003475,0.004653,0.001346,0.001494
min,0.192048,0.204197,0.121159,0.127318,0.045518,0.050676,0.027450,0.028845,0.044825,0.044825,0.018350,0.019282,0.071355,0.075135,0.016735,0.017585,0.051344,0.067314,0.024557,0.025805
25%,0.293848,0.315690,0.180698,0.192925,0.070022,0.080169,0.040167,0.042889,0.069497,0.069497,0.026153,0.027918,0.109162,0.116818,0.025393,0.027112,0.079699,0.105408,0.035415,0.037803
50%,0.294032,0.315817,0.180738,0.192982,0.070077,0.080299,0.040211,0.042938,0.069683,0.069683,0.026219,0.027992,0.109341,0.116875,0.025435,0.027146,0.079809,0.105518,0.035444,0.037842
75%,0.294149,0.315877,0.180806,0.193015,0.070276,0.080429,0.040258,0.042985,0.069748,0.069748,0.026248,0.028026,0.109622,0.117510,0.025450,0.027172,0.079913,0.105579,0.035571,0.037959
max,0.294586,0.316031,0.183966,0.198686,0.070563,0.081065,0.041319,0.044625,0.070296,0.070296,0.027902,0.030135,0.109954,0.119376,0.025578,0.027625,0.080705,0.106042,0.036793,0.039737


In [21]:
#standardization
#Useful: https://stackoverflow.com/questions/40758562/can-anyone-explain-me-standardscaler
scaler = StandardScaler()
std_data = scaler.fit_transform(df_original.values)
df_std = pd.DataFrame(std_data, index=df_original.index, columns=df_original.columns)

In [22]:
df_std.head()

,fA,fT,fC,fG,pAA,pAT,pAC,pAG,pTA,pTA,pTC,pTG,pCA,pCT,pCC,pCG,pGA,pGT,pGC,pGG
">LC521925.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/AI/I-004/2020 RNA, complete genome\n",0.121291,0.143007,0.119374,0.100608,0.222075,0.182822,0.174853,0.153465,0.142188,0.142188,0.009376,-0.006576,0.184350,0.124380,0.142777,0.124009,0.122746,0.094099,0.272297,0.246279
">LC522972.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/KY/V-029/2020 RNA, complete genome\n",0.127135,0.134478,0.108280,0.118074,0.099760,0.097565,0.079189,0.090234,0.082247,0.082247,-0.211769,-0.189173,0.188793,0.058435,0.212970,0.219329,0.219826,0.128630,0.307218,0.309706
">LC522973.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-012/2020 RNA, complete genome\n",0.127135,0.134478,0.108280,0.118074,0.099760,0.087684,0.092234,0.102825,0.072030,0.072030,-0.211769,-0.189173,0.212102,0.047164,0.212970,0.219329,0.219826,0.151962,0.261078,0.265308
">LC522974.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-501/2020 RNA, complete genome\n",0.127135,0.132027,0.112858,0.118074,0.099760,0.085350,0.096956,0.102825,0.074444,0.074444,-0.204181,-0.186521,0.207753,0.051880,0.212970,0.215034,0.219826,0.137915,0.289019,0.287507
">LC522975.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-521/2020 RNA, complete genome\n",0.127135,0.134478,0.112858,0.113929,0.099760,0.097565,0.096956,0.098558,0.082247,0.082247,-0.206943,-0.193511,0.207753,0.054265,0.212970,0.211013,0.223784,0.144192,0.293581,0.287507


In [31]:
#calc distances
#useful: https://www.drawingfromdata.com/making-a-pairwise-distance-matrix-with-pandas
pairwise = pd.DataFrame(
    squareform(pdist(df_original)),
    columns = df_original.index,
    index = df_original.index
)

In [32]:
pairwise

,">LC521925.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/AI/I-004/2020 RNA, complete genome",">LC522972.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/KY/V-029/2020 RNA, complete genome",">LC522973.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-012/2020 RNA, complete genome",">LC522974.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-501/2020 RNA, complete genome",">LC522975.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-521/2020 RNA, complete genome",">LC528232.1 Severe acute respiratory syndrome coronavirus 2 SARS-CoV-2/Hu/DP/Kng/19-020 RNA, complete genome",">LC528233.1 Severe acute respiratory syndrome coronavirus 2 SARS-CoV-2/Hu/DP/Kng/19-027 RNA, complete genome",">LR757995.1 Wuhan seafood market pneumonia virus genome assembly, chromosome: whole_genome",">LR757996.1 Wuhan seafood market pneumonia virus genome assembly, chromosome: whole_genome",">LR757997.1 Wuhan seafood market pneumonia virus genome assembly, chromosome: whole_genome",...,">MT123290.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/IQTC01/human/2020/CHN, complete genome",">MT123291.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/IQTC02/human/2020/CHN, complete genome",">MT123292.1 Severe acute respiratory syndrome-related coronavirus isolate SARS-CoV-2/IQTC04/human/2020/CHN, complete genome",">MT123293.1 Severe acute respiratory syndrome-related coronavirus isolate SARS-CoV-2/IQTC03/human/2020/CHN, complete genome",">MT126808.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/SP02/human/2020/BRA, complete genome",">MT135041.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/105/human/2020/CHN, complete genome",">MT135042.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/231/human/2020/CHN, complete genome",">MT135043.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/233/human/2020/CHN, complete genome",">MT135044.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/235/human/2020/CHN, complete genome",">MT152824.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/WA2/human/2020/USA, complete genome"
">LC521925.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/AI/I-004/2020 RNA, complete genome\n",0.000000,0.000858,0.000931,0.000902,0.000876,0.000775,0.001399,0.001300,0.001580,0.199752,...,0.000866,0.001596,0.001452,0.001606,0.000866,0.001195,0.001195,0.001166,0.001195,0.001660
">LC522972.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/KY/V-029/2020 RNA, complete genome\n",0.000858,0.000000,0.000196,0.000137,0.000134,0.000934,0.001857,0.000940,0.002060,0.199606,...,0.000940,0.001889,0.002076,0.002180,0.000959,0.001188,0.001188,0.001157,0.001188,0.001926
">LC522973.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-012/2020 RNA, complete genome\n",0.000931,0.000196,0.000000,0.000101,0.000108,0.001032,0.001895,0.000928,0.002091,0.199618,...,0.001026,0.001953,0.002162,0.002236,0.001013,0.001192,0.001192,0.001154,0.001192,0.001926
">LC522974.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-501/2020 RNA, complete genome\n",0.000902,0.000137,0.000101,0.000000,0.000081,0.000985,0.001888,0.000929,0.002069,0.199603,...,0.000979,0.001911,0.002122,0.002209,0.001009,0.001192,0.001192,0.001158,0.001192,0.001927
">LC522975.1 Severe acute respiratory syndrome coronavirus 2 2019-nCoV/Japan/TY/WK-521/2020 RNA, complete genome\n",0.000876,0.000134,0.000108,0.000081,0.000000,0.000976,0.001844,0.000929,0.002080,0.199632,...,0.000985,0.001936,0.002100,0.002197,0.000991,0.001190,0.001190,0.001156,0.001190,0.001941
">LC528232.1 Severe acute respiratory syndrome coronavirus 2 SARS-CoV-2/Hu/DP/Kng/19-020 RNA, complete genome\n",0.000775,0.000934,0.001032,0.000985,0.000976,0.000000,0.001768,0.001338,0.001902,0.199770,...,0.000253,0.001693,0.001762